In [ ]:
%pip install opencv-python
%pip install torch
%pip install --upgrade --force-reinstall h5py

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch

# Load Fashion-MNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Define class labels
class_names = ['T-shirt/top', 'Pants', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Preprocess the data
train_images = train_images / 255.0
test_images = test_images / 255.0

# Display information about the dataset
print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Test images shape:", test_images.shape)
print("Test labels shape:", test_labels.shape)

In [ ]:
import matplotlib.pyplot as plt

# Visualize sample images
plt.figure(figsize=(10, 10))
for i in range(25):
    plt.subplot(5, 5, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()


In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load Fashion-MNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Define class labels
class_names = ['T-shirt/top', 'Pants', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

# Preprocess the data
train_images = train_images[..., tf.newaxis] / 255.0
test_images = test_images[..., tf.newaxis] / 255.0

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
)

# Define the model architecture
model = models.Sequential([
    layers.Input(shape=(28, 28, 1)),  # Input layer with specified shape
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Reshape the data for CNN input
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))

# Train the model with data augmentation
history = model.fit(datagen.flow(train_images, train_labels, batch_size=32),
                    steps_per_epoch=int(len(train_images) / 100), epochs=50,
                    validation_data=(test_images, test_labels))

# Plot accuracy and loss
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='test accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Model Accuracy')
plt.show()

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='test loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Model Loss')
plt.show()


In [ ]:
# Function to preprocess the uploaded image
def preprocess_image(image):
    resized_image = cv2.resize(image, (28, 28))
    if len(resized_image.shape) == 3:
        if resized_image.shape[2] == 3:  # Color image
            resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
        resized_image = resized_image / 255.0
        # Add batch and channel dimensions
        preprocessed_image = resized_image[np.newaxis, ..., np.newaxis]
    else:  # Grayscale image
        resized_image = resized_image / 255.0
        # Add batch and channel dimensions
        preprocessed_image = resized_image[np.newaxis, ..., np.newaxis]
    return preprocessed_image

# Function to predict outfit
def predict_outfit(image):
    preprocessed_image = preprocess_image(image)
    print("Preprocessed image shape:", preprocessed_image.shape)  # Debugging: Check the shape of the preprocessed image
    prediction = model(torch.tensor(preprocessed_image, dtype=torch.float32))
    print("Prediction shape:", prediction.shape)  # Debugging: Check the shape of the prediction
    predicted_class = tf.argmax(prediction, axis=1).numpy()[0]
    predicted_label = class_names[predicted_class]
    return predicted_label

# Function to get complementary items for the predicted class
def get_complementary_items(predicted_label):
    complementary_items = {
        'T-shirt/top': ['Jeans', 'Sneakers'],
        'Pullover': ['Shirt', 'Jeans', 'Sneakers'],
        'Shirt': ['Jeans', 'Sneakers', 'Necklace'],
        'Dress': ['Shirt', 'Boots'],
        'Coat': ['Jeans', 'Sneakers'],
        'Sandal': ['Necklace', 'Handbag'],
        'Pants': ['Shirt', 'Sneakers'],
        'Sneaker': ['T-shirt/top', 'Jeans'],
        'Bag': ['Sandals', 'Necklace'],
        'Ankle boot': ['Shirt', 'Sneakers']
    }
    return complementary_items.get(predicted_label, [])


# Function to display outfit suggestions
def display_outfit_suggestions(predicted_label, outfit_suggestions):
    if outfit_suggestions:
        print("Based on the", predicted_label, "you uploaded, we suggest you wear:")
        for suggestion in outfit_suggestions:
            print("-", suggestion)
    else:
        print("Sorry, we couldn't provide outfit suggestions for this image.")

        
# Function to extract dominant colors using K-means clustering
def extract_dominant_colors(image, k=3):
    # Reshape image to a 2D array of pixels
    pixels = image.reshape((-1, 3))
    pixels = np.float32(pixels)

    # Define criteria and apply K-means
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 0.85)  # Adjusting epsilon
    _, labels, centers = cv2.kmeans(pixels, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)

    # Convert centers to uint8 and return
    centers = np.uint8(centers)
    return centers

# Function to recognize color category
def recognize_color(centers):
    # Define color categories (you can extend this)
    color_categories = {
        (0, 0, 255): 'Red',
        (0, 255, 0): 'Green',
        (255, 0, 0): 'Blue',
        (255, 255, 255): 'White',
        (0, 0, 0): 'Black',
        (255, 255, 0): 'Yellow',
        (255, 0, 255): 'Magenta',
        (0, 255, 255): 'Cyan',
        (128, 0, 0): 'Maroon',
        (0, 128, 0): 'Green (Dark)',
        (0, 0, 128): 'Navy',
        (128, 128, 0): 'Olive',
        (128, 0, 128): 'Purple',
        (0, 128, 128): 'Teal',
        (128, 128, 128): 'Gray',
        (192, 192, 192): 'Silver',
        (128, 128, 255): 'Light Blue',
        (128, 0, 255): 'Violet',
        (255, 128, 128): 'Pink',
        (128, 255, 128): 'Light Green',
        (255, 128, 0): 'Orange',
        (255, 0, 128): 'Rose',
        (0, 128, 255): 'Sky Blue',
        (0, 255, 128): 'Spring Green',
        (255, 128, 255): 'Orchid',
        (255, 255, 128): 'Beige',
        (128, 255, 255): 'Turquoise',
        (255, 128, 128): 'Coral',
        (128, 255, 128): 'Mint',
        (128, 128, 255): 'Periwinkle',
        (255, 128, 255): 'Fuchsia',
        (255, 255, 128): 'Peach',
        (128, 255, 255): 'Aquamarine',
        (255, 128, 192): 'Salmon',
        (255, 192, 128): 'Apricot',
        (192, 255, 128): 'Lime',
        (128, 192, 255): 'Light Purple',
        (192, 128, 255): 'Lavender',
        (255, 192, 255): 'Lilac',
        (255, 255, 192): 'Light Yellow',
        (192, 255, 255): 'Sky',
        (255, 192, 192): 'Rose Pink',
        (192, 255, 192): 'Light Green',
        (192, 192, 255): 'Light Purple',
        (255, 255, 255): 'White',
        (0, 0, 0): 'Black'
    }

    # Find the closest color category to each center
    recognized_colors = []
    for center in centers:
        closest_color = min(color_categories, key=lambda x: np.linalg.norm(np.array(x) - center))
        recognized_colors.append(color_categories[closest_color])

    return recognized_colors

# Function to suggest complementary colors based on recognized color
def suggest_complementary_colors(recognized_colors):
    complementary_colors = {
    'Red': ['Cyan', 'Green', 'Blue'],
    'Green': ['Magenta', 'Red', 'Blue'],
    'Blue': ['Yellow', 'Red', 'Green'],
    'Yellow': ['Blue', 'Magenta', 'Cyan'],
    'Magenta': ['Green', 'Yellow', 'Cyan'],
    'Cyan': ['Red', 'Magenta', 'Yellow'],
    'White': ['Black'],
    'Black': ['White'],
    'Orange': ['Blue', 'Purple'],
    'Purple': ['Yellow', 'Orange'],
    'Pink': ['Green', 'Turquoise'],
    'Turquoise': ['Pink', 'Orange'],
}


    return complementary_colors.get(recognized_colors[0], [])

### Upload Your Own Clothing Image 

In [ ]:
# Load the image from the specified file path
file_path = r"USER/Image.png" #REPLACE WITH YOUR USER IMAGE PATH HERE
image = cv2.imread(file_path)

# Check if the image was loaded successfully
if image is not None:
    # Process the loaded image
    predicted_label = predict_outfit(image)
    outfit_suggestions = get_complementary_items(predicted_label)
    display_outfit_suggestions(predicted_label, outfit_suggestions)
    # Extract dominant colors
    dominant_colors = extract_dominant_colors(image)

    # Recognize color
    recognized_colors = recognize_color(dominant_colors)

    # Suggest complementary colors
    complementary_colors = suggest_complementary_colors(recognized_colors)

    # Display suggestions
    print("Recognized color:", recognized_colors[0])
    print("Complementary colors:", complementary_colors)
else:
    print("Error: Unable to load the image from the specified path.")